In [42]:
%load_ext sagemaker_studio_analytics_extension.magics
%sm_analytics emr connect --verify-certificate False --cluster-id j-1OJGFWLF2O1O8 --auth-type None --language python  

The sagemaker_studio_analytics_extension.magics extension is already loaded. To reload it, use:
  %reload_ext sagemaker_studio_analytics_extension.magics
Successfully read emr cluster(j-1OJGFWLF2O1O8) details
Initiating EMR connection..
Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
12,application_1729270775885_0014,pyspark,idle,Link,Link,None,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.
{"namespace": "sagemaker-analytics", "cluster_id": "j-1OJGFWLF2O1O8", "error_message": null, "success": true, "service": "emr", "operation": "connect"}


## Project Distributed Inference Job Testing
### Requirement
- 1. Launch your EMR cluster with the necessary bootstrap package installations.
- 2. Pre-install the required packages.

In [35]:
from pyspark.sql import SparkSession
from common.inference import InferTask

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [36]:
model_version = "v2.0.1"
model, mode = "p2p_model", "infer"
config_dir_path = "/home/hadoop/opt/p2p_model/config/"
model_dir_path = "/home/hadoop/opt/p2p_model/model/"
# Run the inference pipeline
# TODO: Tune spark configuration
spark = (
    SparkSession.builder.appName(f"Inference - {model} - {model_version}")
    .config("spark.sql.shuffle.partitions", "400")
    .config("spark.driver.cores", "1")
    .config("spark.driver.memory", "4G")
    .config("spark.executor.cores", "4")
    .config("spark.executor.memory", "12G")
    .config("spark.sql.parquet.compression.codec", "snappy")
    .config("spark.sql.parquet.datetimeRebaseModeInRead", "LEGACY")
    .config("spark.sql.parquet.datetimeRebaseModeInWrite", "LEGACY")
    .config("spark.sql.parquet.int96RebaseModeInRead", "LEGACY")
    .config("spark.sql.parquet.int96RebaseModeInWrite", "LEGACY")
    .enableHiveSupport()
    .getOrCreate()
)


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [37]:
param = {'run_date':'2020-04-01', 'platform':'stage',}

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [38]:
spark_distributed = InferTask(
    param['run_date'], param['platform'], model, mode, config_dir_path, model_dir_path, model_version
)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [39]:
spark_distributed.run_inference()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

s3://gd-revrelvnc-stage-pltv2/table/pltv2_combine_features/
2020-04-01
load data with size 4079014
target column list ['f2p_1_month_propensity_pct', 'f2p_2_month_propensity_pct']
root
 |-- shopper_id: string (nullable = true)
 |-- customer_id: string (nullable = true)
 |-- prediction_type: string (nullable = true)
 |-- f2p_1_month_propensity_pct: decimal(8,7) (nullable = true)
 |-- f2p_2_month_propensity_pct: decimal(8,7) (nullable = true)
 |-- model_version: string (nullable = false)
 |-- etl_build_mst_ts: date (nullable = true)

None
total data count 4073462
save total data after filter null id count 4073462
+----------+--------------------+---------------+--------------------------+--------------------------+-------------+----------------+
|shopper_id|         customer_id|prediction_type|f2p_1_month_propensity_pct|f2p_2_month_propensity_pct|model_version|etl_build_mst_ts|
+----------+--------------------+---------------+--------------------------+--------------------------+---------